# Qdrant

## 개요 및 특징

- Qdrant는 고성능 벡터 검색을 위한 오픈소스 벡터 데이터베이스이다. 딥러닝 모델에서 추출한 임베딩 벡터를 저장하고, 유사도 기반 검색을 빠르게 수행할 수 있도록 설계되었다. 
- 주요 특징

  - **고성능 검색**: HNSW 기반의 Approximate Nearest Neighbor(ANN) 알고리즘을 사용하여 빠른 벡터 검색 제공
  - **Payload Filtering**: 메타데이터 기반의 정교한 필터링 기능 제공 (예: 나이, 태그 등 조건 지정 가능)
  - **분산 지원**: 샤딩, 레플리카 등의 분산 기능으로 확장성 확보
  - **Python, Javascript, Rust, Java 등 다양한 언어지원**: 시스템 통합, 개발 편의성, 다양한 애플리케이션 구축이 가능.
  - **REST API 및 gRPC 지원**: 다양한 클라이언트 환경에 대응 가능
  - **스냅샷 및 영속성 보장**: 내장된 스냅샷과 WAL 기능으로 데이터 백업과 복구를 지원.
  - **오픈소스 프로젝트**: **Apache 2.0 라이선스** 하에 배포되는 오픈소스 프로젝트로 기업 환경에서도 제약 없이 자유롭게 사용할 수 있으며, 수정 및 배포도 허용된다.
- 사이트
  - 홈페이지: https://qdrant.tech
  - GitHub: https://github.com/qdrant/qdrant

## Qdrant vs 다른 벡터 DB 비교

| 항목            | Qdrant               | FAISS           | Milvus             | Weaviate  |
| ------------- | -------------------- | --------------- | ------------------ | --------- |
| 검색 방식         | HNSW, Flat           | Flat, IVF, HNSW | IVF, HNSW, DiskANN | HNSW      |
| 필터링 지원        | ✅ (Boolean + Nested) | ❌               | ✅                  | ✅         |
| REST API      | ✅                    | ❌               | ✅                  | ✅         |
| gRPC 지원       | ✅                    | ❌               | ✅                  | ❌         |
| 벡터 + 메타데이터 통합 | ✅                    | ❌               | ✅                  | ✅         |
| 클라우드 서비스      | ✅ (Qdrant Cloud)     | ❌               | ✅                  | ✅         |
| 커뮤니티 및 문서     | 활발                   | 활발              | 활발                 | 활발        |
| 설치 용이성        | Docker로 간단           | 로컬 빌드 필요        | Docker 가능          | Docker 가능 |


# Qdrant 아키텍처

## Core Engine 구조

Qdrant의 Core Engine은 다음과 같은 구성 요소로 이루어집니다:

- **Search Engine**
  - HNSW 기반의 벡터 유사도 탐색 수행
- **Segment**
  - Segment는 QDrant 내부의 핵심 저장 및 검색 단위이다.
  - 데이터는 여러 Segment로 나뉘어 저장되며, 하나의 Collection은 여러 Segment로 구성된다.
  - 각 Segment는 **독립적인 인덱스와 저장 공간**을 가진다. 이를 통해 **검색의 정확도, 속도, 저장 효율성**을 높인다.
- **Index Manager**
  - 인덱스 상태, 설정, 생성 및 최적화를 담당
- **Write-Ahead Log (WAL)**
  - 데이터 변경작업(삽입, 수정, 삭제)가 발생하면 **디스크에 먼저 기록하고 이후 실제 적용하는 방식**
  - 장애 발생시 WAL에 기록된 작업들을 재적용해서 데이터 복원이 가능

## Payload System (Meta-Data Storage)
- Qdrant는 각 벡터와 함께 JSON 형태의 payload를 저장할 수 있다. 
    - payload는 meta-data를 말한다.
- payload는 다음과 같은 다양한 타입을 지원:
    - string, integer, float, boolean, list
- payload는 **검색 시 필터링 조건**으로 활용 가능

## Filtering Mechanism
- Query 시 payload 조건을 지정하여 검색 범위를 제한 가능하여 검색 정확도를 높일 수있다.
- 다양한 검색 조건 비교를 지원하고 **nested 필터**를 통해 Payload의 중첩된 JSON 구조 내부의 필드에 대한 조건에 대해서도 필터링 가능하다.

## WAL(Write-Ahead Log) 및 Durability

- 모든 변경 사항(삽입, 삭제 등)은 WAL에 먼저 기록됨
- WAL은 디스크 기반이며 장애 발생 시 복구에 사용
- WAL + Snapshot을 함께 사용해 데이터 무결성과 내구성 확보할 수있다.s


# Qdrant 핵심 구성 요소

## Collection

- Collection은 Qdrant에서 벡터와 payload를 묶어 저장하는 **논리적 데이터베이스 단위** 이다. 
- SQL의 테이블과 유사한 개념으로, 하나의 Collection에는 동일한 차원의 벡터들이 저장된다.

## 주요 설정 항목
- `name`: 컬렉션 이름
- `vectors.size`: 임베딩 벡터 차원 수 (예: 768, 384 등)
- `vectors.distance`: 유사도 계산 방식 (Cosine, Dot, Euclidean)

## Point
- Point는 Qdrant에서 하나의 벡터 데이터 항목을 의미한다. 하나의 Point는 다음 세 가지로 구성된다.
    1. `id`: 고유 식별자 (정수 또는 UUID)
    2. `vector`: 임베딩된 고차원 벡터 값 (예: [0.12, 0.23, ..., 0.91])
    3. `payload`: 해당 벡터와 연결된 메타데이터 (JSON format)

### 사용 예시 (Python SDK)

```python
client.upsert(
  collection_name="my_collection",
  points=[
    PointStruct(
      id=1,
      vector=[0.1, 0.2, 0.3],   # vector 크기는 collection 생성 시 지정한 vector size 와 동일해야 한다.
      payload={"category": "book", "price":20000}
    )
  ]
)
```

### Payload
- Payload는 Point에 연결된 **메타데이터 정보**로, 검색 필터링시 사용된다. JSON 형식으로 저장되며, 다양한 데이터타입을 지원한다.
- Payload의 개별 원소는 **Field** 라고 하며 **key-value** 쌍으로 구성된다.
- 지원 데이터 타입
    - `string`, `number`, `boolean`, `array`, `nested object`

#### 예시

```json
{
  "category": "book",
  "rating": 4.5,
  "tags": ["fiction", "bestseller"],
  "user": {
    "name": "Alice",
    "age": 30
  }
}
```

#### 필터링에서의 활용

```python
condition_category = FieldCondition(
    key="category",
    match=MatchValue(value="book")
)

condition_user_age = FieldCondition(
    key="user.age",     # nested object
    range=Range(gte=25)
)

```

# Qdrant 설치 및 실행

## Python
- `pip install qdrant-client`
- **Local 컴퓨터의 메모리, HDD/SSD**를 이용해 데이터를 관리할 경우 추가 설치 없이 python lib를 이용해 연결할 수있다.

```python
from qdrant_client import QdrantClient
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance, for testing, CI/CD
# OR
client = QdrantClient(path="path/to/db")  # Persists changes to disk, fast prototyping
```

## Server-Client

### Docker로 QDrant 서버 실행
- 이미지 다운로드 및 실행
    - 6333: REST API 포트
    - 6334: gRPC 포트

```bash
docker pull qdrant/qdrant
docker run -p 6333:6333 -p 6334:6334 qdrant/qdrant
docker run -p 6333:6333 -p 6334:6334 -v "qdrant_storage:/qdrant/storage:z"  qdrant/qdrant
```
- 파이썬에서 연결

```python
from qdrant_client import QdrantClient
client = QdrantClient(url="http://localhost:6333")
```
## Qdrant Cloud
- https://cloud.qdrant.io
- 무료 tier 제공 (1GB 용량)
- 웹 UI로 Collection 생성 및 벡터 관리 가능
- 연결

```python
client = QdrantClient(
    url=url,
    api_key=api_key,
)
```


# 실습

## 연결, collection 생성

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

# Client 초기화 - Qdrant 서버와 연결.

# client = QdrantClient(":memory:") # In-Memory 로 연결.

# client = QdrantClient(path="vector_storage/qdrant/test") # Local File로 저장.

# docker run -p 6333:6333 -p 6334:6334 -v "qdrant_storage:/qdrant/storage:z"  qdrant/qdrant
client = QdrantClient(host="localhost", port=6333)

# Collection 생성 - DB/Table(Point(데이터) 저장공간)
COLLECTION_NAME = "test_collection"

# 특정 collection 있는 지 확인
#    있으면 삭제
if client.collection_exists(collection_name=COLLECTION_NAME): # 있는지 여부 확인
    client.delete_collection(collection_name=COLLECTION_NAME) # 삭제

client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(  # Vector 저장소에 대한 설정
        size=4,  # 벡터 차원수 (embedding model 출력 차원수에 맞춰준다.)
        distance=Distance.COSINE  # 코사인유사도, EUCLID(유클리이안), DOT(내적)
    )
)

# 컬렉션 조회
print(client.get_collections())

# client.close() #연결 종료

collections=[CollectionDescription(name='test_collection')]


## 데이터 삽입
- `upsert()`

In [14]:
from qdrant_client.models import  PointStruct 
# PointStruct:  개별 Point(데이터) 생성 (id, vector, payload)
dataset = [
        PointStruct(id=1, vector=[0.05, 0.61, 0.76, 0.74], payload={"city": "Berlin", "country":"Germany", "num":1000}),
        PointStruct(id=2, vector=[0.19, 0.81, 0.75, 0.11], payload={"city": "London", "country":"England", "num":5000}),
        PointStruct(id=3, vector=[0.36, 0.55, 0.47, 0.94], payload={"city": "Moscow", "country":"Russia", "num":6000}),
        PointStruct(id=4, vector=[0.18, 0.01, 0.85, 0.80], payload={"city": "New York", "country":"USA", "num":10000}),
        PointStruct(id=5, vector=[0.28, 0.03, 0.15, 0.10], payload={"city": "LA", "country":"USA", "num":20000}),
        PointStruct(id=6, vector=[0.24, 0.18, 0.22, 0.44], payload={"city": "Beijing", "country":"China", "num":15000}),
        PointStruct(id=7, vector=[0.35, 0.08, 0.11, 0.44], payload={"city": "Mumbai", "country":"India", "num":16000}),
        PointStruct(id=8, vector=[0.49, 0.77, 0.75, 0.31], payload={"city": "Liverpool", "country":"England", "num":17000}),
        PointStruct(id=9, vector=[0.43, 0.72, 0.12, 0.38], payload={"city": "Shanghai", "country":"China", "num":123000}),
]
upsert_info = client.upsert(
    collection_name=COLLECTION_NAME,
    points=dataset, 
    wait=True  # Upsert가 완전히 적용된 후에 결과를 응답: True, Upsert 로그만 작성하고 응답:False
)
print(upsert_info)


operation_id=1 status=<UpdateStatus.COMPLETED: 'completed'>


## 검색

### 유사도 검색
- `query_points()`

In [ ]:
search_result = client.query_points(
    collection_name=COLLECTION_NAME,
    query=[0.2, 0.1, 0.9, 1.2], # query(질문)의 embedding vector
    limit=3, # top-k 
)

for point in search_result.points:
    print(point.id, point.score, point.payload, sep=" , ")
    print("=======================================")

4 , 0.98368233 , {'city': 'New York', 'country': 'USA', 'num': 10000}
6 , 0.90634227 , {'city': 'Beijing', 'country': 'China', 'num': 15000}
3 , 0.892581 , {'city': 'Moscow', 'country': 'Russia', 'num': 6000}


In [17]:
search_result

QueryResponse(points=[ScoredPoint(id=4, version=1, score=0.98368233, payload={'city': 'New York', 'country': 'USA', 'num': 10000}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=6, version=1, score=0.90634227, payload={'city': 'Beijing', 'country': 'China', 'num': 15000}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=3, version=1, score=0.892581, payload={'city': 'Moscow', 'country': 'Russia', 'num': 6000}, vector=None, shard_key=None, order_value=None)])

## Filtering

- Qdrant의 Filtering은 **벡터 검색 전에 payload(메타데이터)에 대해 조건을 먼저 적용**하는 기능이다.
- Filtering 적용시 검색 흐름(순서)

  1. Payload 조건으로 후보 벡터를 먼저 필터링
  2. 필터링된 벡터에 대해서만 유사도 검색 수행

- Filtering의 효과.

  - 검색 정확도 향상 (불필요한 벡터 제거)
  - 검색 속도 향상 (탐색 공간 축소)
  - 현실적인 검색 조건(가격, 지역, 카테고리, 상태 등)과 벡터 검색을 결합하여 검색할 수 있다.
  - RAG, 추천 시스템, 하이브리드 검색의 필수 구성 요소

- 다양한 예제: - https://qdrant.tech/documentation/concepts/filtering/

### Filter의 전체 구조 (Filter Clauses)

- 여러 조건을 논리 연산으로 결합하는 구조이다.
- 논리 연산자
  
| 절            | 의미                     | 논리 연산 대응   |
| ------------ | ---------------------- | ---------- |
| `must`       | 모든 조건을 반드시 만족         | AND        |
| `should`     | 조건 중 하나 이상 만족          | OR         |
| `min_should` | `should` 중 최소 k개 이상 만족 | OR + 최소 개수 |
| `must_not`   | 해당 조건을 절대 만족하면 안 됨     | NOT        |

- **이 네 가지 구조는 자유롭게 조합 및 중첩 가능하다.**
- 예시

  ```python
  query_filter = Filter(
      must=[
          # category == "book"
          FieldCondition(
              key="category",
              match=MatchValue(value="book")
          ),

          # title match_text "러닝"
          FieldCondition(
              key="title",
              match=MatchText("러닝")
          ),
      ],
      must_not=[
          # is_soldout == true 인 데이터 제외
          FieldCondition(
              key="is_soldout",
              match=MatchValue(value=True)
          )
      ]
  )
  ```
- “카테고리는 book이고 title에 러닝이 들어가면서, 품절되지 않은(is_soldout 이 true가 아닌) 상품만 검색”


### 조건 타입 (Condition Types)

- Filtering에서 실제로 사용하는 **payload 조건 연산자들**이다.
- **연산자들**
  -  `match` 계열 (값 비교 조건)
    -  정확 일치 (`=`)

        ```python
        condition = FieldCondition(
            key="brand",
            match=MatchValue(value="Nike")
        )

        ```

  - `match_any`
    -  여러 값 중 하나 일치 (IN)

          ```python
          condition = FieldCondition(
              key="color",
              match_any=MatchAny(values=["red", "blue"])
          )
          ```
  - `match_except` 
    - 특정 값들 제외 (NOT IN)

      ```python
      condition = FieldCondition(
          key="status",
          match_except=MatchExcept(values=["deleted"])
      )
      ```
  - `match_text`
    - 텍스트 검색. 지정한 payload 값을 단어(토큰) 단위로 분해한 뒤, 해당 단어가 포함된 문서를 검색한다.

      ```python
        condition = FieldCondition(
            key="description",
            match=MatchText("스니커즈")
        )
        ```
    - **토큰 기반 inverted index를 사용하는 full-text 검색**
      - 예)
        ```bash
        문서 1: "이 신발은 가볍고 편안한 러닝화입니다"
        문서 2: "이 신발은 겨울용 부츠입니다"
        문서 3: "편안한 슬리퍼입니다"
        ```
        단어 단위로 토큰화한 뒤 각 단어가 어느 문서에 있는지를 index화 한다.
        ```bash
        "신발은"     → [1, 2]
        "가볍고"     → [1]
        "편안한"     → [1, 3]
        "러닝화입니다" → [1]
        "겨울용"     → [2]
        "부츠입니다" → [2]
        "슬리퍼입니다" → [3]
        ```


  - `range` 
    - 수치형, 날짜형 데이터에 대해 **범위 조건**을 설정한다.
    - 연산자
      | 연산자   | 의미 |
      | ----- | -- |
      | `gt`  | 초과 |
      | `gte` | 이상 |
      | `lt`  | 미만 |
      | `lte` | 이하 |

        ```python
        condition = FieldCondition(
            key="price",
            range=Range(gte=10000, lt=20000)
        )
        ```

  -  `exists`
     - key가 존재 하는 지 여부
     - `condition = HasFieldCondition(key="price")` : price key가 있는지 여부

  - `is_empty`
    - key에 값이 비어있는가?
    - key는 있는데 거기에 **값이 비어있는지 여부**
    - `condition = IsEmptyCondition(key="reviews")`: reviews key는 존재하지만 값이 [] 이거나 'null' 인 데이터를 검색. (빈문자열은 false-버전에 따라 다름.)

  - `values_count`
    - 배열형 payload의 **요소 개수(길이)** 조건을 지정.

      ```python
          condition = FieldCondition(
            key="tags",
            values_count=ValuesCount(gte=2)
          )
      ```
    -  태그가 2개 이상인 데이터만 검색
    
  -  `geo`
     -  위경도 좌표 payload에 대해 사용하는 **거리 기반 필터링**이다.

          ```python
            condition = FieldCondition(
                key="location",
                geo_radius=GeoRadius(
                    center=GeoPoint(lat=37.5, lon=127.0),
                    radius=3000
                )
            )
          ```

### Filtering과 Payload Index
- Payload Index 없는 field에 대해 Filtering을 하면 기본적으로 모든 데이터에 대해 조건 검색을 하게 된다(Full Scan). 거기에 유사도 검색까지 하게 되면 검색이 너무 오래 걸리게 된다.
- 이 문제를 해결하기 위해 Payload에 index를 걸어 줄 수 있다. Payload index는 필드 타입에 따라 B-Tree, 해시 인덱스, inverted index 등 다양한 내부 인덱스 구조를 사용하여 빠르게 검색할 수 있도록 해준다.
- 다음 필드들에 index를 만들어 주는 것이 좋다.
  - 자주 filtering 하는 숫자 필드 (price, age, score)
  - 범주 값을 저장하는 필드 (category, type, status)
  - 날짜 (created_at, updated_at)
  - 위치 (geo index)
  - boolean 값을 가지는 필드
- payload 설정
  ```python
  client.create_payload_index(
      collection_name="products",
      field_name="price",   # field name
      field_schema=PayloadSchemaType.INTEGER  # field의 데이터 타입. 타입에 맞는 index 알고리즘을 사용한다.
  )
  ```
- field type
  - `PayloadSchemaType.KEYWORD` - 문자열. 일치 검색일 때 지정
  - `PayloadSchemaType.TEXT` - 문자열. match_text (특정 단어를 포함하고 있는지를 검색일 때 지정)
  - `PayloadSchemaType.INTEGER`
  - `PayloadSchemaType.FLOAT`
  - `PayloadSchemaType.GEO`
  - `PayloadSchemaType.BOOL`
  - `PayloadSchemaType.DATETIME`
  - `PayloadSchemaType.UUID`
  - 문자열 타입의 필드
    - `match`(==), `match_any`, `match_except` 비교를 주로 할 경우 `PayloadSchemaType.KEYWORD` 로 설정한다.
      - 범주형 field에 설정. (카테고리, 상태값, 타입, 태그등)
    - `match_text` 비교를 주로 할 경우 `PayloadSchemaType.TEXT`로 설정한다.
      - 긴 문서 field에 설정한다. (설명, 리뷰내용 등)

In [21]:
from qdrant_client.models import (
    Filter, # 필터를 만들때 사용
    FieldCondition, # 개별 조건
    MatchValue, # == 일치 검색에 사용할 값
    MatchText,  # 부분일치 검색에 사용할 값
    Range,      # 범위(>< >= <=) 검색에 사용할 값
    PayloadSchemaType # Payload Field에 Index를 걸어줄 때 사용.
)

In [22]:
# Payload Index 설정 - 자주 Filtering에 사용되는 field에 적용.
client.create_payload_index(
    collection_name=COLLECTION_NAME,
    field_name="country", # 어느 필드에 index를 걸어줄지.
    field_schema=PayloadSchemaType.KEYWORD  # 문자열 - 일치 비교를 하는 경우.
)
client.create_payload_index(
    collection_name=COLLECTION_NAME,
    field_name="num",
    field_schema=PayloadSchemaType.INTEGER # 정수
)

UpdateResult(operation_id=5, status=<UpdateStatus.COMPLETED: 'completed'>)

In [23]:
# 확인
from pprint import pprint

# 특정 collection의 정보 조회
info = client.get_collection(COLLECTION_NAME)
pprint(info.payload_schema)

{'country': PayloadIndexInfo(data_type=<PayloadSchemaType.KEYWORD: 'keyword'>, params=None, points=9),
 'num': PayloadIndexInfo(data_type=<PayloadSchemaType.INTEGER: 'integer'>, params=None, points=9)}


In [31]:
############################### 
# 필터를 이용한 검색
#
# 1. payload filtering
# 2. vector 유사도

search_result = client.query_points(
    collection_name=COLLECTION_NAME, 
    query=[0.2, 0.3, 0.9, 0.7], # 질문의 Embedding Vector
    limit=3, # TOP-K
    query_filter=Filter(  # Payload Filter 설정
        must=[
            FieldCondition(
                key="country",
                match=MatchValue(value="USA")
            )
        ]
    )
)

In [32]:
for point in search_result.points:
    print(point.id, point.payload, point.score, sep=" , ")

4 , {'city': 'New York', 'country': 'USA', 'num': 10000} , 0.9657378
5 , {'city': 'LA', 'country': 'USA', 'num': 20000} , 0.6752659


In [39]:
search_result = client.query_points(
    collection_name=COLLECTION_NAME,
    query=[0.3, 0.7, 0.2, 0.9],
    limit=3,
    query_filter=Filter(
        must=[
            FieldCondition(
                key="num",
                # range=Range(gt=10000)  # num > 10000
                range=Range(gt=10000, lt=20000) # 10000 < num < 20000
            )
        ]
    )
)
for point in search_result.points:
    print(point.id, point.payload, point.score, sep=" , ")

6 , {'city': 'Beijing', 'country': 'China', 'num': 15000} , 0.92594165
7 , {'city': 'Mumbai', 'country': 'India', 'num': 16000} , 0.8370426
8 , {'city': 'Liverpool', 'country': 'England', 'num': 17000} , 0.76345015


In [43]:
search_result = client.query_points(
    collection_name=COLLECTION_NAME,
    query=[0.3, 0.7, 0.2, 0.9],
    limit=3,
    query_filter=Filter(
        must=[
            FieldCondition(
                key="num",
                # range=Range(gt=10000)  # num > 10000
                range=Range(gt=10000, lt=20000) # 10000 < num < 20000
            ),
            FieldCondition(
                key="city",
                match=MatchText(text="Be")   # 부분일치
            )
        ]
    )
)
for point in search_result.points:
    print(point.id, point.payload, point.score, sep=" , ")

6 , {'city': 'Beijing', 'country': 'China', 'num': 15000} , 0.92594165


In [44]:
search_result = client.query_points(
    collection_name=COLLECTION_NAME,
    query=[0.3, 0.7, 0.2, 0.9],
    limit=3,
    query_filter=Filter(
        should=[
            FieldCondition(
                key="num",
                # range=Range(gt=10000)  # num > 10000
                range=Range(gt=10000, lt=20000) # 10000 < num < 20000
            ),
            FieldCondition(
                key="city",
                match=MatchText(text="Be")   # 부분일치
            )
        ]
    )
)
for point in search_result.points:
    print(point.id, point.payload, point.score, sep=" , ")

6 , {'city': 'Beijing', 'country': 'China', 'num': 15000} , 0.92594165
1 , {'city': 'Berlin', 'country': 'Germany', 'num': 1000} , 0.8603711
7 , {'city': 'Mumbai', 'country': 'India', 'num': 16000} , 0.8370426


In [47]:
from qdrant_client.models import MinShould

search_result = client.query_points(
    collection_name=COLLECTION_NAME,
    query=[0.7, 0.2, 0.5, 0.8],
    limit=3,
    query_filter=Filter(
        min_should=MinShould(    # 조건들 중에 K개를 만족
            min_count=2,    # 다음 조건중 최소 2개가 True point들
            conditions=[
                FieldCondition(key="country", match=MatchValue(value="USA")),
                FieldCondition(key="city", match=MatchText(text="Be")),
                FieldCondition(key="num", range=Range(lt=5000))
            ] 
        )
    )
)
for point in search_result.points:
    print(point.id, point.payload, point.score, sep=" , ")

6 , {'city': 'Beijing', 'country': 'China', 'num': 15000} , 0.96997607
5 , {'city': 'LA', 'country': 'USA', 'num': 20000} , 0.89598995
4 , {'city': 'New York', 'country': 'USA', 'num': 10000} , 0.84763557


## 단순 조회 
- `scroll()`

In [48]:
# 조회결과: (조회결과: list, 다음Field의 id(next offset))
result, next_field_id = client.scroll(
    collection_name=COLLECTION_NAME,
    limit=3     # 순서대로 3개 조회
)

print("다음 ID:", next_field_id)
for r in result:
    print(r.id, r.payload)

다음 ID: 4
1 {'city': 'Berlin', 'country': 'Germany', 'num': 1000}
2 {'city': 'London', 'country': 'England', 'num': 5000}
3 {'city': 'Moscow', 'country': 'Russia', 'num': 6000}


In [49]:
result2, next_field_id2 = client.scroll(
    collection_name=COLLECTION_NAME,
    offset=next_field_id, # 어디서 부터 가져올지 ID값 지정.
    limit=3               # offset부터 limit개를 조회
)

print("다음 ID:", next_field_id2)
for r in result2:
    print(r.id, r.payload)

다음 ID: 7
4 {'city': 'New York', 'country': 'USA', 'num': 10000}
5 {'city': 'LA', 'country': 'USA', 'num': 20000}
6 {'city': 'Beijing', 'country': 'China', 'num': 15000}


In [52]:
next_id = None
while True:
    result, next_id = client.scroll(
        collection_name=COLLECTION_NAME,
        offset=next_id,
        limit=4
    )
    print(len(result))
    if next_id == None:
        break

4
4
1


In [54]:
# filter 적용. Filter -> Scroll
result, next_id = client.scroll(
    collection_name=COLLECTION_NAME,
    limit=5,
    scroll_filter=Filter(
        must=[FieldCondition(key="country", match=MatchValue(value="England"))]
    )
)
print(next_id)
result

None


[Record(id=2, payload={'city': 'London', 'country': 'England', 'num': 5000}, vector=None, shard_key=None, order_value=None),
 Record(id=8, payload={'city': 'Liverpool', 'country': 'England', 'num': 17000}, vector=None, shard_key=None, order_value=None)]

## 삭제
- `delete()`

In [55]:
client.delete(
    collection_name=COLLECTION_NAME,
    points_selector=[2, 8]  # 삭제할 ID를 지정.
)

UpdateResult(operation_id=6, status=<UpdateStatus.COMPLETED: 'completed'>)

In [56]:
result, _ = client.scroll(
    collection_name=COLLECTION_NAME   # limit 기본값: 10
)
result

[Record(id=1, payload={'city': 'Berlin', 'country': 'Germany', 'num': 1000}, vector=None, shard_key=None, order_value=None),
 Record(id=3, payload={'city': 'Moscow', 'country': 'Russia', 'num': 6000}, vector=None, shard_key=None, order_value=None),
 Record(id=4, payload={'city': 'New York', 'country': 'USA', 'num': 10000}, vector=None, shard_key=None, order_value=None),
 Record(id=5, payload={'city': 'LA', 'country': 'USA', 'num': 20000}, vector=None, shard_key=None, order_value=None),
 Record(id=6, payload={'city': 'Beijing', 'country': 'China', 'num': 15000}, vector=None, shard_key=None, order_value=None),
 Record(id=7, payload={'city': 'Mumbai', 'country': 'India', 'num': 16000}, vector=None, shard_key=None, order_value=None),
 Record(id=9, payload={'city': 'Shanghai', 'country': 'China', 'num': 123000}, vector=None, shard_key=None, order_value=None)]

In [58]:
# Payload Filter로 삭제
client.delete(
    collection_name=COLLECTION_NAME,
    points_selector=Filter(  # Filter 조건으로 삭제
        must=[FieldCondition(key="country", match=MatchValue(value="Russia"))]
    )
)

UpdateResult(operation_id=7, status=<UpdateStatus.COMPLETED: 'completed'>)

In [59]:
result, _ = client.scroll(
    collection_name=COLLECTION_NAME   # limit 기본값: 10
)
result

[Record(id=1, payload={'city': 'Berlin', 'country': 'Germany', 'num': 1000}, vector=None, shard_key=None, order_value=None),
 Record(id=4, payload={'city': 'New York', 'country': 'USA', 'num': 10000}, vector=None, shard_key=None, order_value=None),
 Record(id=5, payload={'city': 'LA', 'country': 'USA', 'num': 20000}, vector=None, shard_key=None, order_value=None),
 Record(id=6, payload={'city': 'Beijing', 'country': 'China', 'num': 15000}, vector=None, shard_key=None, order_value=None),
 Record(id=7, payload={'city': 'Mumbai', 'country': 'India', 'num': 16000}, vector=None, shard_key=None, order_value=None),
 Record(id=9, payload={'city': 'Shanghai', 'country': 'China', 'num': 123000}, vector=None, shard_key=None, order_value=None)]

## 수정

In [60]:
# Point(데이터)를 변경 -> 같은 ID의 PointStruct객체를 upsert하면 데이터를 변경.

p = PointStruct(
    id=9, 
    vector=[1.2, 0.3, -0.7, 2.4], 
    payload={"city":"Seoul", "Country":"Korea", "num":500000, "info":"대한민국의 수도"}
)
# 데이터 추가
client.upsert(
    collection_name=COLLECTION_NAME,
    points=[p]  # 이미 있는 ID의 point가 upsert -> 변경.
)

UpdateResult(operation_id=8, status=<UpdateStatus.COMPLETED: 'completed'>)

In [61]:
client.scroll(collection_name=COLLECTION_NAME)

([Record(id=1, payload={'city': 'Berlin', 'country': 'Germany', 'num': 1000}, vector=None, shard_key=None, order_value=None),
  Record(id=4, payload={'city': 'New York', 'country': 'USA', 'num': 10000}, vector=None, shard_key=None, order_value=None),
  Record(id=5, payload={'city': 'LA', 'country': 'USA', 'num': 20000}, vector=None, shard_key=None, order_value=None),
  Record(id=6, payload={'city': 'Beijing', 'country': 'China', 'num': 15000}, vector=None, shard_key=None, order_value=None),
  Record(id=7, payload={'city': 'Mumbai', 'country': 'India', 'num': 16000}, vector=None, shard_key=None, order_value=None),
  Record(id=9, payload={'city': 'Seoul', 'Country': 'Korea', 'num': 500000, 'info': '대한민국의 수도'}, vector=None, shard_key=None, order_value=None)],
 None)

In [62]:
# Payload를 변경.
## Payload의 특정 field의 값을 변경 - set_payload()
client.set_payload(
    collection_name=COLLECTION_NAME,
    payload={"city":"Chungching", "num":65000, "info":"중국의 도시"}, # 변경, 추가.
    points=[6] # 변경할 Field들의 ID
)

UpdateResult(operation_id=9, status=<UpdateStatus.COMPLETED: 'completed'>)

In [63]:
client.scroll(collection_name=COLLECTION_NAME)

([Record(id=1, payload={'city': 'Berlin', 'country': 'Germany', 'num': 1000}, vector=None, shard_key=None, order_value=None),
  Record(id=4, payload={'city': 'New York', 'country': 'USA', 'num': 10000}, vector=None, shard_key=None, order_value=None),
  Record(id=5, payload={'city': 'LA', 'country': 'USA', 'num': 20000}, vector=None, shard_key=None, order_value=None),
  Record(id=6, payload={'city': 'Chungching', 'country': 'China', 'num': 65000, 'info': '중국의 도시'}, vector=None, shard_key=None, order_value=None),
  Record(id=7, payload={'city': 'Mumbai', 'country': 'India', 'num': 16000}, vector=None, shard_key=None, order_value=None),
  Record(id=9, payload={'city': 'Seoul', 'Country': 'Korea', 'num': 500000, 'info': '대한민국의 수도'}, vector=None, shard_key=None, order_value=None)],
 None)

In [65]:
# payload 자체를 변경. - overwrite_payload()
client.overwrite_payload(
    collection_name=COLLECTION_NAME,
    payload={"도시이름":"인천", "나라":"대한민국"},
    points=[4]
)

UpdateResult(operation_id=11, status=<UpdateStatus.COMPLETED: 'completed'>)

In [66]:
client.scroll(collection_name=COLLECTION_NAME)

([Record(id=1, payload={'city': 'Berlin', 'country': 'Germany', 'num': 1000}, vector=None, shard_key=None, order_value=None),
  Record(id=4, payload={'도시이름': '인천', '나라': '대한민국'}, vector=None, shard_key=None, order_value=None),
  Record(id=5, payload={'city': 'LA', 'country': 'USA', 'num': 20000}, vector=None, shard_key=None, order_value=None),
  Record(id=6, payload={'city': 'Chungching', 'country': 'China', 'num': 65000, 'info': '중국의 도시'}, vector=None, shard_key=None, order_value=None),
  Record(id=7, payload={'city': 'Mumbai', 'country': 'India', 'num': 16000}, vector=None, shard_key=None, order_value=None),
  Record(id=9, payload={'city': 'Seoul', 'Country': 'Korea', 'num': 500000, 'info': '대한민국의 수도'}, vector=None, shard_key=None, order_value=None)],
 None)

In [67]:
# Payload의 특정 Field를 삭제 - delete_payload
client.delete_payload(
    collection_name=COLLECTION_NAME,
    keys=['city', 'num'], # 삭제할 field의 key 들
    points=[7, 6] # 삭제할 id들
)
client.scroll(collection_name=COLLECTION_NAME)

([Record(id=1, payload={'city': 'Berlin', 'country': 'Germany', 'num': 1000}, vector=None, shard_key=None, order_value=None),
  Record(id=4, payload={'도시이름': '인천', '나라': '대한민국'}, vector=None, shard_key=None, order_value=None),
  Record(id=5, payload={'city': 'LA', 'country': 'USA', 'num': 20000}, vector=None, shard_key=None, order_value=None),
  Record(id=6, payload={'info': '중국의 도시', 'country': 'China'}, vector=None, shard_key=None, order_value=None),
  Record(id=7, payload={'country': 'India'}, vector=None, shard_key=None, order_value=None),
  Record(id=9, payload={'city': 'Seoul', 'Country': 'Korea', 'num': 500000, 'info': '대한민국의 수도'}, vector=None, shard_key=None, order_value=None)],
 None)

In [68]:
# payload 자체를 삭제 - clear_payload
client.clear_payload(
    collection_name=COLLECTION_NAME,
    points_selector=[1]
)
client.scroll(collection_name=COLLECTION_NAME)

([Record(id=1, payload={}, vector=None, shard_key=None, order_value=None),
  Record(id=4, payload={'도시이름': '인천', '나라': '대한민국'}, vector=None, shard_key=None, order_value=None),
  Record(id=5, payload={'city': 'LA', 'country': 'USA', 'num': 20000}, vector=None, shard_key=None, order_value=None),
  Record(id=6, payload={'info': '중국의 도시', 'country': 'China'}, vector=None, shard_key=None, order_value=None),
  Record(id=7, payload={'country': 'India'}, vector=None, shard_key=None, order_value=None),
  Record(id=9, payload={'city': 'Seoul', 'Country': 'Korea', 'num': 500000, 'info': '대한민국의 수도'}, vector=None, shard_key=None, order_value=None)],
 None)